In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd drive/My Drive/Colab Notebooks/ 
!ls

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks
CIS521	CIS700	  cvna.csv		     Prof_Matni
CIS680	COVID_19  FNCE206FinalProject.ipynb


In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as si
import datetime
from scipy.stats import norm
import numpy as np 
import math 

In [3]:
# parameters: 
risk_free_rate = 0.0073 

## import data

In [4]:
cvna_data = pd.read_csv('cvna.csv')
cvna_data.head()

,tradedate,ticker,stockprice,expiration,lastpricecall,callvolume,callopeninterest,strike,lastpriceput,putvolume,putopeninterest
0,2020-10-01,CVNA,237.71,2020-10-16,0.0,0,0,85.0,0.00,0,52
1,2020-10-01,CVNA,237.71,2020-10-16,0.0,0,0,90.0,0.07,30,254
2,2020-10-01,CVNA,237.71,2020-10-16,0.0,0,0,95.0,0.00,0,22
3,2020-10-01,CVNA,237.71,2020-10-16,0.0,0,0,100.0,0.00,0,127
4,2020-10-01,CVNA,237.71,2020-10-16,0.0,0,0,105.0,0.00,0,42


In [5]:
cvna_data['tradedate']= pd.to_datetime(cvna_data['tradedate'])
cvna_data['expiration']= pd.to_datetime(cvna_data['expiration'])
cvna_data = cvna_data.drop(columns=['ticker', 'callopeninterest','putopeninterest'])
cvna_data.head()

,tradedate,stockprice,expiration,lastpricecall,callvolume,strike,lastpriceput,putvolume
0,2020-10-01,237.71,2020-10-16,0.0,0,85.0,0.00,0
1,2020-10-01,237.71,2020-10-16,0.0,0,90.0,0.07,30
2,2020-10-01,237.71,2020-10-16,0.0,0,95.0,0.00,0
3,2020-10-01,237.71,2020-10-16,0.0,0,100.0,0.00,0
4,2020-10-01,237.71,2020-10-16,0.0,0,105.0,0.00,0


## Choose the two expiry dates 

In [6]:
# cvna_data.groupby(cvna_data.expiration)
start_date = '2021-01-01 00:00:00'
agg_vols_by_expiry = cvna_data.groupby(['expiration'])["callvolume", "putvolume"].apply(lambda x : x.sum()).reset_index()
agg_vols_by_expiry['allvolume'] = agg_vols_by_expiry.callvolume + agg_vols_by_expiry.putvolume
agg_vols_by_expiry = agg_vols_by_expiry.loc[agg_vols_by_expiry['expiration'] >= start_date]
agg_vols_by_expiry.sort_values(by = 'allvolume', ascending = False,inplace = True)
agg_vols_by_expiry.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,expiration,callvolume,putvolume,allvolume
14,2021-01-15,12139,19875,32014
16,2021-05-21,19424,5666,25090
15,2021-02-19,3251,5607,8858
17,2022-01-21,1298,790,2088
18,2023-01-20,106,28,134


In [7]:
expiry_date_1 = list(agg_vols_by_expiry.expiration)[0]  
expiry_date_2 = list(agg_vols_by_expiry.expiration)[1]  

cvna_expiry_date_1 = cvna_data[cvna_data.expiration == expiry_date_1]
print('len(cvna_expiry_date_1), ', len(cvna_expiry_date_1))
cvna_expiry_date_1.head() 

len(cvna_expiry_date_1),  2033


,tradedate,stockprice,expiration,lastpricecall,callvolume,strike,lastpriceput,putvolume
373,2020-10-01,237.71,2021-01-15,0.0,0,15.0,0.08,3
374,2020-10-01,237.71,2021-01-15,0.0,0,18.0,0.00,0
375,2020-10-01,237.71,2021-01-15,0.0,0,20.0,0.00,0
376,2020-10-01,237.71,2021-01-15,0.0,0,23.0,0.19,1
377,2020-10-01,237.71,2021-01-15,0.0,0,25.0,0.00,0


In [8]:
cvna_expiry_date_2 = cvna_data[cvna_data.expiration == expiry_date_2]
print('len(cvna_expiry_date_2), ', len(cvna_expiry_date_2))
cvna_expiry_date_2.head() 

len(cvna_expiry_date_2),  1443


,tradedate,stockprice,expiration,lastpricecall,callvolume,strike,lastpriceput,putvolume
678,2020-10-01,237.71,2021-05-21,0.0,0,85.0,4.6,1
679,2020-10-01,237.71,2021-05-21,0.0,0,90.0,0.0,0
680,2020-10-01,237.71,2021-05-21,0.0,0,95.0,0.0,0
681,2020-10-01,237.71,2021-05-21,0.0,0,100.0,0.0,0
682,2020-10-01,237.71,2021-05-21,0.0,0,105.0,0.0,0


## Get prelim data

In [9]:
# change here for the two expiry dates 
data = cvna_expiry_date_1
expiry_date = expiry_date_1

In [10]:
prelim_start_date = '2020-10-01 00:00:00'
prelim_end_date = '2020-10-13 00:00:00'
prelim_data = data.loc[(data['tradedate'] >= prelim_start_date) & (data['tradedate'] <= prelim_end_date)]
print('len(prelim_data)', len(prelim_data))

len(prelim_data) 493


In [11]:
prelim_data['abs_strike_minus_stock'] = abs(prelim_data.strike - prelim_data.stockprice)
prelim_data['days_til_expiration'] = (prelim_data.expiration - prelim_data.tradedate).dt.components.days
prelim_data['years_til_expiration'] = prelim_data.days_til_expiration / 365
temp = prelim_data[prelim_data['abs_strike_minus_stock'].isin(prelim_data.groupby('tradedate').min()['abs_strike_minus_stock'].values)]
temp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,tradedate,stockprice,expiration,lastpricecall,callvolume,strike,lastpriceput,putvolume,abs_strike_minus_stock,days_til_expiration,years_til_expiration
416,2020-10-01,237.71,2021-01-15,36.80,10,240.0,41.20,22,2.29,106,0.290411
1320,2020-10-02,222.60,2021-01-15,40.15,47,220.0,42.05,33,2.60,105,0.287671
2152,2020-10-05,225.00,2021-01-15,43.50,5,220.0,36.50,7,5.00,102,0.279452
2153,2020-10-05,225.00,2021-01-15,38.95,19,230.0,41.85,51,5.00,102,0.279452
2264,2020-10-06,214.54,2021-01-15,0.00,0,210.0,34.30,1,4.54,101,0.276712
3136,2020-10-07,216.96,2021-01-15,0.00,0,220.0,37.80,3,3.04,100,0.273973
4176,2020-10-08,213.99,2021-01-15,37.00,9,210.0,34.00,4,3.99,99,0.271233
4302,2020-10-09,213.29,2021-01-15,35.80,3,210.0,32.20,1,3.29,98,0.268493
5231,2020-10-12,209.12,2021-01-15,33.00,4,210.0,32.80,4,0.88,95,0.260274
5970,2020-10-13,216.77,2021-01-15,33.45,1024,220.0,36.35,1007,3.23,94,0.257534


## Find IV 

In [215]:
def find_vol(target_value, call_put, S, K, T, r):
    MAX_ITERATIONS = 100
    PRECISION = 1.0e-5

    sigma = 0.5
    for i in range(MAX_ITERATIONS):
        price = bs_price(call_put, S, K, T, r, sigma)
        vega = bs_vega(call_put, S, K, T, r, sigma)

        price = price
        diff = target_value - price  # our root

        # print(i, sigma, diff)

        if (abs(diff) < PRECISION):
            return sigma
        sigma = sigma + diff/vega # f(x) / f'(x)

    # value wasn't found, return best guess so far
    return sigma

n = norm.pdf
N = norm.cdf

def bs_price(cp_flag,S,K,T,r,v,q=0.0):
    d1 = (np.log(S/K)+(r+v*v/2.)*T)/(v*np.sqrt(T))
    d2 = d1-v*np.sqrt(T)
    if cp_flag == 'c':
        price = S*np.exp(-q*T)*N(d1)-K*np.exp(-r*T)*N(d2)
    else:
        price = K*np.exp(-r*T)*N(-d2)-S*np.exp(-q*T)*N(-d1)
    return price

def bs_vega(cp_flag,S,K,T,r,v,q=0.0):
    d1 = (np.log(S/K)+(r+v*v/2.)*T)/(v*np.sqrt(T))
    return S * np.sqrt(T)*n(d1)

In [216]:
call_IV_holder, put_IV_holder, day_IV_holder, total_vol_per_day = [], [], [], [] 
for row in range(len(temp)): 
  print('day: ', list(temp.tradedate)[row])
  K = list(temp.strike)[row]
  T = list(temp.years_til_expiration)[row]
  S = list(temp.stockprice)[row]
  r = risk_free_rate 
  
  # call IV 
  V_market = list(temp.lastpricecall)[row]
  cp = 'c' # call option
  call_IV = find_vol(V_market, cp, S, K, T, r)
  call_IV_holder.append(call_IV)
  print('Implied vol: %.2f%%' % (call_IV * 100))

  print('Market price = %.2f' % V_market)
  print('Model price = %.2f' % bs_price(cp, S, K, T, r, call_IV))

  # put IV 
  V_market = list(temp.lastpriceput)[row]
  cp = 'p' # put option
  put_IV = find_vol(V_market, cp, S, K, T, r)
  put_IV_holder.append(put_IV)
  print('Implied vol: %.2f%%' % (put_IV * 100))

  print('Market price = %.2f' % V_market)
  print('Model price = %.2f' % bs_price(cp, S, K, T, r, put_IV))

  call_vol = list(temp.callvolume)[row]
  put_vol = list(temp.putvolume)[row]
  weighed_IV_today = (call_vol*call_IV+put_vol*put_IV)/(call_vol+put_vol) if call_vol+put_vol else (call_IV+put_IV)/2
  day_IV_holder.append(weighed_IV_today)
  print('put/call weighed IV: ', weighed_IV_today)
  total_vol_per_day.append(call_vol+put_vol)
  print('\n---------------------------------------------\n')


total_weighed_IV = sum([day_IV_holder[i]*total_vol_per_day[i] for i in range(len(temp))]) / sum(total_vol_per_day)
print('total weighed IV for ATM option that expires on {} is {}'.format(expiry_date, round(total_weighed_IV,5)))

implied_volatility = total_weighed_IV

day:  2020-10-01 00:00:00
Implied vol: 73.95%
Market price = 36.80
Model price = 36.80
Implied vol: 79.16%
Market price = 41.20
Model price = 41.20
put/call weighed IV:  0.7753527441445711

---------------------------------------------

day:  2020-10-02 00:00:00
Implied vol: 82.27%
Market price = 40.15
Model price = 40.15
Implied vol: 93.05%
Market price = 42.05
Model price = 42.05
put/call weighed IV:  0.8671565314047438

---------------------------------------------

day:  2020-10-05 00:00:00
Implied vol: 87.64%
Market price = 43.50
Model price = 43.50
Implied vol: 84.24%
Market price = 36.50
Model price = 36.50
put/call weighed IV:  0.8565932817748063

---------------------------------------------

day:  2020-10-05 00:00:00
Implied vol: 86.67%
Market price = 38.95
Model price = 38.95
Implied vol: 83.17%
Market price = 41.85
Model price = 41.85
put/call weighed IV:  0.8411972234667114

---------------------------------------------

day:  2020-10-06 00:00:00
Implied vol: -1.08%
Market

## trading on 10/14

In [224]:
# first find 3-4 strikes surrounding the stock price on 10/14 
stock_price_10_14 = list(data[data.tradedate == '2020-10-14 00:00:00'].stockprice)[0]
print('On 10/14, stock price = ', stock_price_10_14)

data['abs_strike_minus_stock'] = abs(data.strike - data.stockprice)
data['days_til_expiration'] = (data.expiration - data.tradedate).dt.components.days
data['years_til_expiration'] = data.days_til_expiration / 365
temp_trade = data[data.tradedate == '2020-10-14 00:00:00'].sort_values(by = 'abs_strike_minus_stock').head(3)
temp_trade

On 10/14, stock price =  213.22


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

,tradedate,stockprice,expiration,lastpricecall,callvolume,strike,lastpriceput,putvolume,abs_strike_minus_stock,days_til_expiration,years_til_expiration
7034,2020-10-14,213.22,2021-01-15,31.67,7,210.0,32.36,4,3.22,93,0.254795
7035,2020-10-14,213.22,2021-01-15,30.00,10,220.0,36.00,4,6.78,93,0.254795
7033,2020-10-14,213.22,2021-01-15,36.00,1,200.0,26.17,4,13.22,93,0.254795


In [225]:
def compute_fair_price_call(row): 
  S = row.stockprice 
  K = row.strike 
  cp='c'
  T = row.years_til_expiration
  fair_price = bs_price(cp, S, K, T, risk_free_rate, implied_volatility)
  return fair_price

def compute_fair_price_put(row): 
  S = row.stockprice 
  K = row.strike 
  cp='p'
  T = row.years_til_expiration
  fair_price = bs_price(cp, S, K, T, risk_free_rate, implied_volatility)
  return fair_price

temp_trade['fair_price_call'] = temp_trade.apply(compute_fair_price_call,axis=1)
temp_trade['fair_price_put'] = temp_trade.apply(compute_fair_price_put,axis=1)
temp_trade['call_is_u_p'] = temp_trade.lastpricecall < temp_trade.fair_price_call
temp_trade['put_is_u_p'] = temp_trade.lastpriceput < temp_trade.fair_price_put
temp_trade.head()

,tradedate,stockprice,expiration,lastpricecall,callvolume,strike,lastpriceput,putvolume,abs_strike_minus_stock,days_til_expiration,years_til_expiration,fair_price_call,fair_price_put,call_is_u_p,put_is_u_p
7034,2020-10-14,213.22,2021-01-15,31.67,7,210.0,32.36,4,3.22,93,0.254795,35.626564,32.016327,True,False
7035,2020-10-14,213.22,2021-01-15,30.00,10,220.0,36.00,4,6.78,93,0.254795,31.494533,37.865713,True,True
7033,2020-10-14,213.22,2021-01-15,36.00,1,200.0,26.17,4,13.22,93,0.254795,40.222511,26.630856,True,True


In [226]:
# buy if u/p, sell if o/p 
# then we find the delta of each of the three options 
def get_price_delta_call(row):
    # call delta 
    s = row.stockprice
    k = row.strike 
    t = row.years_til_expiration
    rf=risk_free_rate
    div=0
    vol=implied_volatility

    d1 = (math.log(s/k)+(rf+div+math.pow(vol,2)/2)*t)/(vol*math.sqrt(t))
    d2 = d1-vol*math.sqrt(t)
    calc_price = norm.cdf(d1)*s*math.exp(-div*t)-norm.cdf(d2)*k*math.exp(-rf*t)   
    call_delta = norm.cdf(d1)

    return call_delta 

def get_price_delta_put(row):
    # call delta 
    s = row.stockprice
    k = row.strike 
    t = row.years_til_expiration
    rf=risk_free_rate
    div=0
    vol=implied_volatility

    d1 = (math.log(s/k)+(rf+div+math.pow(vol,2)/2)*t)/(vol*math.sqrt(t))
    d2 = d1-vol*math.sqrt(t)

    # put delta 
    calc_price = -norm.cdf(-d1)*s*math.exp(-div*t)+norm.cdf(-d2)*k * math.exp(-rf*t)
    put_delta = -norm.cdf(-d1) 

    return put_delta

def total_delta_position(row): 
  call_sign = 1 if row.call_is_u_p else -1 
  put_sign =  1 if row.put_is_u_p else -1 
  return call_sign * row.call_delta + put_sign * row.put_delta 

def gamma(S, K, T, r=risk_free_, sigma):
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    prob_density = 1 / np.sqrt(2 * np.pi) * np.exp(-d1 ** 2 * 0.5)
    
    gamma = prob_density / (S * sigma * np.sqrt(T))
    
    return gamma

In [227]:
temp_trade['call_delta'] = temp_trade.apply(get_price_delta_call,axis=1)
temp_trade['put_delta'] = temp_trade.apply(get_price_delta_put,axis=1)
temp_trade['delta_position'] = temp_trade.apply(total_delta_position,axis=1)
temp_trade.sort_values(by = 'strike',inplace=True)
temp_trade

,tradedate,stockprice,expiration,lastpricecall,callvolume,strike,lastpriceput,putvolume,abs_strike_minus_stock,days_til_expiration,years_til_expiration,fair_price_call,fair_price_put,call_is_u_p,put_is_u_p,call_delta,put_delta,delta_position
7033,2020-10-14,213.22,2021-01-15,36.00,1,200.0,26.17,4,13.22,93,0.254795,40.222511,26.630856,True,True,0.642436,-0.357564,0.284871
7034,2020-10-14,213.22,2021-01-15,31.67,7,210.0,32.36,4,3.22,93,0.254795,35.626564,32.016327,True,False,0.596392,-0.403608,1.000000
7035,2020-10-14,213.22,2021-01-15,30.00,10,220.0,36.00,4,6.78,93,0.254795,31.494533,37.865713,True,True,0.551198,-0.448802,0.102397


In [228]:
print('total delta position = ', sum(list(temp_trade.delta_position)))

total delta position =  1.3872678348387975


## For every Wednesday onwards, do the following: 
1) check if the long option is still U/P and short is still O/P. If not, revert the position

2) revise delta by recalculating delta 

## Trading day 10/21

In [155]:
trade_date = trade_start_date = list(temp_trade.tradedate)[0]
election_date = pd.to_datetime("2020-11-04")
strategy_revise_dates = [] 

while trade_date < election_date: 
  trade_date += datetime.timedelta(weeks=1)
  strategy_revise_dates.append(trade_date) 

if strategy_revise_dates[-1] >= election_date: 
  strategy_revise_dates = strategy_revise_dates[:-1]
  
print('Trade Revision Dates: ', strategy_revise_dates)

Trade Revision Dates:  [Timestamp('2020-10-21 00:00:00'), Timestamp('2020-10-28 00:00:00')]


In [156]:
# first do it on the first trading date
trading_day = strategy_revise_dates[0]
temp_trade = data[(data['strike'].isin(list(temp_trade.strike))) & (data.tradedate == trading_day)]

temp_trade['fair_price_call'] = temp_trade.apply(compute_fair_price_call,axis=1)
temp_trade['fair_price_put'] = temp_trade.apply(compute_fair_price_put,axis=1)
temp_trade['call_is_u_p'] = temp_trade.lastpricecall < temp_trade.fair_price_call
temp_trade['put_is_u_p'] = temp_trade.lastpriceput < temp_trade.fair_price_put
temp_trade['call_delta'] = temp_trade.apply(get_price_delta_call,axis=1)
temp_trade['put_delta'] = temp_trade.apply(get_price_delta_put,axis=1)
temp_trade['delta_position'] = temp_trade.apply(total_delta_position,axis=1)
temp_trade.sort_values(by = 'strike',inplace=True)

temp_trade


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

,tradedate,stockprice,expiration,lastpricecall,callvolume,strike,lastpriceput,putvolume,abs_strike_minus_stock,days_til_expiration,years_til_expiration,fair_price_call,fair_price_put,call_is_u_p,put_is_u_p,call_delta,put_delta,delta_position
9813,2020-10-21,192.22,2021-05-21,42.0,9,200.0,47.42,1,7.78,212,0.580822,42.685423,49.619218,True,True,0.594971,-0.405029,0.189941
9814,2020-10-21,192.22,2021-05-21,0.0,0,210.0,54.80,4,17.78,212,0.580822,39.252800,56.144285,True,True,0.563126,-0.436874,0.126253
9815,2020-10-21,192.22,2021-05-21,36.6,1,220.0,0.00,0,27.78,212,0.580822,36.106930,62.956104,False,True,0.532374,-0.467626,-1.000000


In [157]:
print('total delta position = ', sum(list(temp_trade.delta_position)))

total delta position =  -0.6838059841591997


## Next trading day - 10/28

In [125]:
trading_day = strategy_revise_dates[1]
temp_trade = data[(data['strike'].isin(list(temp_trade.strike))) & (data.tradedate == trading_day)]

temp_trade['fair_price_call'] = temp_trade.apply(compute_fair_price_call,axis=1)
temp_trade['fair_price_put'] = temp_trade.apply(compute_fair_price_put,axis=1)
temp_trade['call_is_u_p'] = temp_trade.lastpricecall < temp_trade.fair_price_call
temp_trade['put_is_u_p'] = temp_trade.lastpriceput < temp_trade.fair_price_put
temp_trade['call_delta'] = temp_trade.apply(get_price_delta_call,axis=1)
temp_trade['put_delta'] = temp_trade.apply(get_price_delta_put,axis=1)
temp_trade['delta_position'] = temp_trade.apply(total_delta_position,axis=1)
temp_trade.sort_values(by = 'strike',inplace=True)

temp_trade

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,tradedate,stockprice,expiration,lastpricecall,callvolume,strike,lastpriceput,putvolume,abs_strike_minus_stock,days_til_expiration,years_til_expiration,fair_price_call,fair_price_put,call_is_u_p,put_is_u_p,call_delta,put_delta,delta_position
13398,2020-10-28,202.36,2021-01-15,31.48,15,200.0,29.35,7,2.36,79,0.216438,31.000179,28.324429,False,False,0.587722,-0.412278,-0.175443
13399,2020-10-28,202.36,2021-01-15,26.57,1,210.0,33.50,10,7.64,79,0.216438,26.862084,34.170546,True,True,0.536035,-0.463965,0.072070
13400,2020-10-28,202.36,2021-01-15,23.60,11,220.0,40.60,10,17.64,79,0.216438,23.214164,40.506838,False,False,0.486166,-0.513834,0.027667


In [126]:
print('total delta position = ', sum(list(temp_trade.delta_position)))

total delta position =  -0.07570615931669


## ELECTION DAY - recalculate IV based on data from 11/04

In [127]:
recalculate_IV_data = data[data.tradedate == election_date]
IV2_table = recalculate_IV_data.sort_values(by='abs_strike_minus_stock').head(1)

IV2_table

,tradedate,stockprice,expiration,lastpricecall,callvolume,strike,lastpriceput,putvolume,abs_strike_minus_stock,days_til_expiration,years_til_expiration
16697,2020-11-04,201.17,2021-01-15,24.26,7,200.0,25.08,3,1.17,72,0.19726


In [128]:
K = list(IV2_table.strike)[0]
T = list(IV2_table.years_til_expiration)[0]
S = list(IV2_table.stockprice)[0]
r = risk_free_rate 
  
# call IV 
V_market = list(IV2_table.lastpricecall)[0]
cp = 'c' # call option
call_IV = find_vol(V_market, cp, S, K, T, r)
print('Implied vol: %.2f%%' % (call_IV * 100))
print('Market price = %.2f' % V_market)
print('Model price = %.2f' % bs_price(cp, S, K, T, r, call_IV))
print('')
# put IV 
V_market = list(IV2_table.lastpriceput)[0]
cp = 'p' # put option
put_IV = find_vol(V_market, cp, S, K, T, r)
print('Implied vol: %.2f%%' % (put_IV * 100))
print('Market price = %.2f' % V_market)
print('Model price = %.2f' % bs_price(cp, S, K, T, r, put_IV))

call_vol = list(IV2_table.callvolume)[0]
put_vol = list(IV2_table.putvolume)[0]
implied_volatility_post_election = (call_vol*call_IV+put_vol*put_IV)/(call_vol+put_vol) if call_vol+put_vol else (call_IV+put_IV)/2

print('total weighed IV for ATM option that expires on {} AFTER ELECTION is {}'.format(expiry_date, round(implied_volatility_post_election,5)))


Implied vol: 66.48%
Market price = 24.26
Model price = 24.26

Implied vol: 72.97%
Market price = 25.08
Model price = 25.08
total weighed IV for ATM option that expires on 2021-01-15 00:00:00 AFTER ELECTION is 0.68425


rule of thumb - its less than before election - so it's good...

## Do the same strategy as before - 11/11

In [129]:
trade_date = election_date
liquidation_date = pd.to_datetime("2020-11-18")
strategy_revise_dates = [] 

while trade_date < liquidation_date: 
  trade_date += datetime.timedelta(weeks=1)
  strategy_revise_dates.append(trade_date) 

if strategy_revise_dates[-1] >= liquidation_date: 
  strategy_revise_dates = strategy_revise_dates[:-1]
  
print('Trade Revision Dates: ', strategy_revise_dates)

Trade Revision Dates:  [Timestamp('2020-11-11 00:00:00')]


In [131]:
def compute_fair_price_call(row): 
  S = row.stockprice 
  K = row.strike 
  cp='c'
  T = row.years_til_expiration
  fair_price = bs_price(cp, S, K, T, risk_free_rate, implied_volatility_post_election)
  return fair_price

def compute_fair_price_put(row): 
  S = row.stockprice 
  K = row.strike 
  cp='p'
  T = row.years_til_expiration
  fair_price = bs_price(cp, S, K, T, risk_free_rate, implied_volatility_post_election)
  return fair_price


# buy if u/p, sell if o/p 
# then we find the delta of each of the three options 
def get_price_delta_call(row):
    # call delta 
    s = row.stockprice
    k = row.strike 
    t = row.years_til_expiration
    rf=risk_free_rate
    div=0
    vol=implied_volatility_post_election

    d1 = (math.log(s/k)+(rf+div+math.pow(vol,2)/2)*t)/(vol*math.sqrt(t))
    d2 = d1-vol*math.sqrt(t)
    calc_price = norm.cdf(d1)*s*math.exp(-div*t)-norm.cdf(d2)*k*math.exp(-rf*t)   
    call_delta = norm.cdf(d1)

    return call_delta 

def get_price_delta_put(row):
    # call delta 
    s = row.stockprice
    k = row.strike 
    t = row.years_til_expiration
    rf=risk_free_rate
    div=0
    vol=implied_volatility_post_election

    d1 = (math.log(s/k)+(rf+div+math.pow(vol,2)/2)*t)/(vol*math.sqrt(t))
    d2 = d1-vol*math.sqrt(t)

    # put delta 
    calc_price = -norm.cdf(-d1)*s*math.exp(-div*t)+norm.cdf(-d2)*k * math.exp(-rf*t)
    put_delta = -norm.cdf(-d1) 

    return put_delta

In [132]:
trading_day = strategy_revise_dates[0]
temp_trade = data[(data['strike'].isin(list(temp_trade.strike))) & (data.tradedate == trading_day)]

temp_trade['fair_price_call'] = temp_trade.apply(compute_fair_price_call,axis=1)
temp_trade['fair_price_put'] = temp_trade.apply(compute_fair_price_put,axis=1)
temp_trade['call_is_u_p'] = temp_trade.lastpricecall < temp_trade.fair_price_call
temp_trade['put_is_u_p'] = temp_trade.lastpriceput < temp_trade.fair_price_put
temp_trade['call_delta'] = temp_trade.apply(get_price_delta_call,axis=1)
temp_trade['put_delta'] = temp_trade.apply(get_price_delta_put,axis=1)
temp_trade['delta_position'] = temp_trade.apply(total_delta_position,axis=1)
temp_trade.sort_values(by = 'strike',inplace=True)

temp_trade


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,tradedate,stockprice,expiration,lastpricecall,callvolume,strike,lastpriceput,putvolume,abs_strike_minus_stock,days_til_expiration,years_til_expiration,fair_price_call,fair_price_put,call_is_u_p,put_is_u_p,call_delta,put_delta,delta_position
20479,2020-11-11,207.33,2021-01-15,25.95,47,200.0,19.3,27,7.33,65,0.178082,27.353126,19.763295,True,True,0.607778,-0.392222,0.215556
20480,2020-11-11,207.33,2021-01-15,21.85,56,210.0,27.2,3,2.67,65,0.178082,22.758455,25.155632,True,False,0.541639,-0.458361,1.000000
20481,2020-11-11,207.33,2021-01-15,16.30,51,220.0,30.0,17,12.67,65,0.178082,18.808852,31.193038,True,True,0.477454,-0.522546,-0.045092


In [133]:
print('total delta position = ', sum(list(temp_trade.delta_position)))

total delta position =  1.1704640186515762


## Liquidation Day 11/18 

In [37]:
# first do it on the first trading date
trading_day = strategy_revise_dates[2]
trading_day_df_4 = data[(data['strike'].isin(list(trading_day_df.strike))) & (data.tradedate == trading_day)]
trading_day_df_4.sort_values(by = 'strike',inplace=True)

trading_day_df_4

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,tradedate,ticker,stockprice,expiration,lastpricecall,callvolume,callopeninterest,strike,lastpriceput,putvolume,putopeninterest,abs_strike_minus_stock,days_til_expiration,years_til_expiration
23344,2020-11-18,CVNA,224.12,2021-01-15,38.85,51,583,200.0,12.05,12,658,24.12,58,0.158904
23345,2020-11-18,CVNA,224.12,2021-01-15,31.65,15,461,210.0,15.55,19,290,14.12,58,0.158904
23346,2020-11-18,CVNA,224.12,2021-01-15,24.80,11,2458,220.0,19.55,40,2212,4.12,58,0.158904


In [139]:
datetime.datetime. strptime('18/11/20', '%d/%m/%y') - datetime.datetime. strptime('11/11/20', '%d/%m/%y')

datetime.timedelta(7)